In [ ]:
%matplotlib notebook

In [ ]:
%%bash
[[ -z "$COLAB_GPU" ]] && exit 0
apt install subversion >> log
pip install pyarts typhon >> log
svn co -q https://arts.mi.uni-hamburg.de/svn/rt/arts-xml-data/trunk/spectroscopy/Perrin/ >> log

In [ ]:
"""Calculate and plot absorption cross sections."""
import re

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from pyarts.workspace import Workspace
from typhon.plots import styles
from typhon.utils import Timer


def plot_absorption(verbosity=0):
    # Define parameters
    species = "H2O"
    temperature = 300
    pressure = 800e2

    # Call ARTS to calculate absorption cross sections
    ws = setup_absxsec(species, verbosity=verbosity)
    freq, abs_xsec = calculate_absxsec(ws, pressure, temperature)

    # Plot the results.
    plt.style.use(styles("typhon"))

    fig, ax = plt.subplots()
    plt.subplots_adjust(bottom=0.25)

    axcolor = "white"
    axtemp = plt.axes([0.2, 0.1, 0.65, 0.03], facecolor=axcolor)
    axpres = plt.axes([0.2, 0.05, 0.65, 0.03], facecolor=axcolor)

    spres = Slider(
        axpres,
        "Pressure",
        0,
        1000,
        valstep=1,
        valfmt="%0.0f hPa",
        valinit=pressure / 100,
    )
    stemp = Slider(
        axtemp,
        "Temperature",
        200,
        400,
        valstep=1,
        valfmt="%0.0f K",
        valinit=temperature,
    )

    (lines,) = ax.plot(freq / 1e9, abs_xsec, rasterized=True)

    def update(_=None):
        p = spres.val * 100
        t = stemp.val
        with Timer():
            freq, xsec = calculate_absxsec(ws, p, t)
        lines.set_ydata(xsec)
        ax.set_title(f"{tag2tex(species)} p:{p / 100:0.0f} hPa T:{t:0.0f} K")
        fig.canvas.draw_idle()

    stemp.on_changed(update)
    spres.on_changed(update)

    ax.set_xlim(freq.min() / 1e9, freq.max() / 1e9)
    ax.set_ylim(bottom=0)
    ax.set_xlabel("Frequency [GHz]")
    ax.set_ylabel(r"Abs. cross section [$\sf m^2$]")
    ax.set_title(f"{tag2tex(species)} p:{pressure/100} hPa T:{temperature:0.0f} K")

    # fig.savefig(  # Save figure.
    #     f"plots/plot_xsec_{species}_{pressure:.0f}Pa_{temperature:.0f}K.pdf"
    # )
    plt.show()  # Open an interactive figure


def tag2tex(tag):
    """Replace all numbers in a species tag with LaTeX subscripts."""
    return re.sub("([a-zA-Z]+)([0-9]+)", r"\1$_{\2}$", tag)


def setup_absxsec(
    species="N2O",
    fmin=10e9,
    fmax=2000e9,
    fnum=10_000,
    lineshape="LP",
    normalization="RQ",
    verbosity=2,
):
    """Setup absorption cross sections.

    Parameters:
        species (str): Absorption species name.
        fmin (float): Minimum frequency [Hz].
        fmax (float): Maximum frequency [Hz].
        fnum (int): Number of frequency grid points.
        lineshape (str): Line shape model.
        normalization (str): Line shape normalization factor.
        verbosity (int): Set ARTS verbosity (``0`` prevents all output).

    Returns:
        Workspace: ARTS Workspace.
    """
    # Create ARTS workspace and load default settings
    ws = Workspace(verbosity=0)
    ws.execute_controlfile("general/general.arts")
    ws.execute_controlfile("general/continua.arts")
    ws.execute_controlfile("general/agendas.arts")
    ws.verbositySetScreen(ws.verbosity, verbosity)

    # We do not want to calculate the Jacobian Matrix
    ws.jacobianOff()

    # Agenda for scalar gas absorption calculation
    ws.Copy(ws.abs_xsec_agenda, ws.abs_xsec_agenda__noCIA)

    # Define absorption species
    ws.abs_speciesSet(species=[species])
    ws.ArrayOfIndexSet(ws.abs_species_active, [0])

    # Line catalogue: Perrin or HITRAN
    ws.ReadSplitARTSCAT(
        basename="./Perrin/",
        fmin=0.9 * fmin,
        fmax=1.1 * fmax,
        globalquantumnumbers="",
        localquantumnumbers="",
        ignore_missing=0,
    )
    ws.abs_lines_per_speciesCreateFromLines()

    # Set the lineshape function for all calculated tags
    ws.abs_linesSetLineShapeType(ws.abs_lines, lineshape)
    ws.abs_linesSetCutoff(ws.abs_lines, "None", 0.0)
    ws.abs_linesSetNormalization(ws.abs_lines, normalization)

    # isotop
    ws.isotopologue_ratiosInitFromBuiltin()

    # Atmospheric settings
    ws.AtmosphereSet1D()

    # Create a frequency grid
    ws.VectorNLinSpace(ws.f_grid, fnum, fmin, fmax)

    return ws


def calculate_absxsec(
    ws, pressure=800e2, temperature=300.0,
):
    """Calculate absorption cross sections.

    Parameters:
        ws (Workspace): ARTS Workspace.
        pressure (float): Atmospheric pressure [Pa].
        temperature (float): Atmospheric temperature [K].

    Returns:
        ndarray, ndarray: Frequency grid [Hz], Abs. cross sections [m^2]
    """
    # Setting the pressure, temperature and vmr
    ws.NumericSet(ws.rtp_pressure, float(pressure))  # [Pa]
    ws.NumericSet(ws.rtp_temperature, float(temperature))  # [K]
    ws.VectorSet(ws.rtp_vmr, np.array([1.0]))  # [VMR]
    ws.Touch(ws.abs_nlte)

    ws.AbsInputFromRteScalars()

    # Calculate absorption cross sections
    ws.lbl_checkedCalc()
    ws.abs_xsec_agenda_checkedCalc()

    ws.abs_xsec_per_speciesInit()
    ws.abs_xsec_per_speciesAddLines()

    return ws.f_grid.value, ws.abs_xsec_per_species.value[0]

In [ ]:
plot_absorption(verbosity=0)